In [77]:
import csv
import cvxpy as cp
import numpy as np
import pandas as pd
import time
import glob

In [89]:
# Load the dataframe
def load_df(file):
    df = pd.read_csv(file)
    df = df.dropna(subset=['Position', 'Name', 'Points', 'Salary'])
   
    def apply_position_transform(r):
        pos = r['Position'].split('/')
        pos_ret = pos
        if 'PG' in pos or 'SG' in pos:
            pos_ret.append('G')
        if 'PF' in pos or 'SF' in pos:
            pos_ret.append('F')
        pos_ret.append('UTIL')
        return pos

    if len(df) > 0:
        df['PosTot'] = df.apply(apply_position_transform, axis=1)
    return df

In [90]:
# Solve the MILP
def solve_milp(df, W=50000):
    # Build the buckets
    positions = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UTIL']
    buckets = np.zeros((len(positions), len(df)))
    for i,pos in enumerate(positions):
        for j,position in enumerate(df['PosTot'].values.tolist()):
            if pos in position:
                buckets[i, j] = 1

    # Get the values
    weights = df['Salary'].values
    values = df['Points'].values

    # The variable we are solving for
    selections = cp.Variable((len(df), len(positions)), boolean=True)

    # The constraints
    constraints = [
        weights@cp.sum(selections, axis=1) <= W,
        buckets[0, :]@selections[:, 0] == 1, # PG position constraint
        buckets[1, :]@selections[:, 1] == 1, # SG position constraint
        buckets[2, :]@selections[:, 2] == 1, # SF position constraint
        buckets[3, :]@selections[:, 3] == 1, # PF position constraint
        buckets[4, :]@selections[:, 4] == 1, # C position constraint
        buckets[5, :]@selections[:, 5] == 1, # G position constraint
        buckets[6, :]@selections[:, 6] == 1, # F position constraint
        buckets[7, :]@selections[:, 7] == 1, # UTIL position constraint
        cp.sum(selections) == len(positions),
        cp.sum(selections, axis=1) <=1
    ]

    # Optimization
    optimization = objective = cp.Maximize(values@cp.sum(selections, axis=1))

    # Solve the problem
    knapsack_problem = cp.Problem(objective, constraints)

    # Output solution
    result = knapsack_problem.solve()
    return selections, result

In [91]:
# Get the final players
def get_results(date, selections, df):
    results, tot_score, tot_value = [date], 0, 0
    for i,row in enumerate(np.transpose(selections.value).tolist()):
        for j,v in enumerate([int(round(v)) for v in row]):
            if v == 1:
                player = df.iloc[j]
                results.extend([player['Name'], player['Points'], player['Salary']])
                tot_score = tot_score + player['Points']
                tot_value = tot_value + player['Salary']
    results.extend([tot_score, tot_value])
    return results

In [92]:
# Get the date
def parse_date(file):
    return '-'.join(file.split('.')[0].split('_')[1:])

In [102]:
# Run this for every valid salary file
files = glob.glob('data/salaries/*')
results = []
for file in files:
    df = load_df(file)
    if len(df) > 0:
        print(file)
        selections, _ = solve_milp(df)
        if not selections.value is None:
            output = get_results(parse_date(file), selections, df)
            results.append(output)
    else:
        print("Bad file: %s....." % file)

data/salaries/DKSalaries_2015_02_15.csv
data/salaries/DKSalaries_2015_02_01.csv
data/salaries/DKSalaries_2015_05_08.csv
data/salaries/DKSalaries_2019_04_28.csv
data/salaries/DKSalaries_2018_02_16.csv
data/salaries/DKSalaries_2019_03_21.csv
data/salaries/DKSalaries_2018_02_02.csv
data/salaries/DKSalaries_2019_04_14.csv
data/salaries/DKSalaries_2018_05_23.csv
data/salaries/DKSalaries_2019_03_09.csv
data/salaries/DKSalaries_2019_04_19.csv
data/salaries/DKSalaries_2019_03_10.csv
data/salaries/DKSalaries_2018_02_27.csv
data/salaries/DKSalaries_2019_03_04.csv
Bad file: data/salaries/DKSalaries_2019_04_25.csv.....
data/salaries/DKSalaries_2018_05_06.csv
data/salaries/DKSalaries_2015_05_11.csv
data/salaries/DKSalaries_2015_05_05.csv
data/salaries/DKSalaries_2016_12_29.csv
data/salaries/DKSalaries_2016_12_01.csv
data/salaries/DKSalaries_2015_02_24.csv
data/salaries/DKSalaries_2016_12_15.csv
data/salaries/DKSalaries_2018_12_22.csv
data/salaries/DKSalaries_2017_04_12.csv
data/salaries/DKSalaries_

data/salaries/DKSalaries_2019_11_17.csv
data/salaries/DKSalaries_2018_10_20.csv
Bad file: data/salaries/DKSalaries_2018_10_08.csv.....
data/salaries/DKSalaries_2016_05_22.csv
data/salaries/DKSalaries_2017_04_15.csv
data/salaries/DKSalaries_2017_03_08.csv
data/salaries/DKSalaries_2017_04_01.csv
data/salaries/DKSalaries_2020_09_17.csv
data/salaries/DKSalaries_2017_03_20.csv
data/salaries/DKSalaries_2016_02_03.csv
data/salaries/DKSalaries_2020_09_03.csv
data/salaries/DKSalaries_2015_12_26.csv
data/salaries/DKSalaries_2018_12_25.csv
data/salaries/DKSalaries_2018_12_19.csv
data/salaries/DKSalaries_2015_05_02.csv
data/salaries/DKSalaries_2016_12_06.csv
data/salaries/DKSalaries_2015_02_23.csv
data/salaries/DKSalaries_2016_12_12.csv
data/salaries/DKSalaries_2019_03_17.csv
data/salaries/DKSalaries_2019_03_03.csv
data/salaries/DKSalaries_2019_04_22.csv
data/salaries/DKSalaries_2018_05_15.csv
data/salaries/DKSalaries_2018_02_08.csv
data/salaries/DKSalaries_2018_05_01.csv
data/salaries/DKSalaries_

data/salaries/DKSalaries_2015_04_19.csv
data/salaries/DKSalaries_2015_04_25.csv
data/salaries/DKSalaries_2019_12_09.csv
data/salaries/DKSalaries_2019_12_21.csv
data/salaries/DKSalaries_2020_08_30.csv
data/salaries/DKSalaries_2014_12_22.csv
data/salaries/DKSalaries_2017_02_07.csv
data/salaries/DKSalaries_2016_03_30.csv
data/salaries/DKSalaries_2020_01_14.csv
data/salaries/DKSalaries_2017_02_13.csv
data/salaries/DKSalaries_2016_03_24.csv
data/salaries/DKSalaries_2020_08_24.csv
data/salaries/DKSalaries_2016_04_05.csv
data/salaries/DKSalaries_2020_01_28.csv
data/salaries/DKSalaries_2016_03_18.csv
data/salaries/DKSalaries_2020_08_18.csv
data/salaries/DKSalaries_2016_04_11.csv
data/salaries/DKSalaries_2016_03_19.csv
data/salaries/DKSalaries_2016_04_10.csv
data/salaries/DKSalaries_2020_08_19.csv
data/salaries/DKSalaries_2020_01_29.csv
data/salaries/DKSalaries_2016_03_25.csv
data/salaries/DKSalaries_2017_02_12.csv
data/salaries/DKSalaries_2020_01_01.csv
data/salaries/DKSalaries_2020_08_25.csv


data/salaries/DKSalaries_2019_05_02.csv
data/salaries/DKSalaries_2018_03_28.csv
data/salaries/DKSalaries_2018_04_21.csv
data/salaries/DKSalaries_2019_05_16.csv
data/salaries/DKSalaries_2018_04_09.csv
data/salaries/DKSalaries_2019_02_23.csv
data/salaries/DKSalaries_2018_03_14.csv
data/salaries/DKSalaries_2015_01_06.csv
data/salaries/DKSalaries_2016_11_23.csv
data/salaries/DKSalaries_2015_01_12.csv
data/salaries/DKSalaries_2017_10_28.csv
data/salaries/DKSalaries_2018_01_05.csv
data/salaries/DKSalaries_2018_01_11.csv
data/salaries/DKSalaries_2015_11_17.csv
data/salaries/DKSalaries_2015_11_03.csv
data/salaries/DKSalaries_2016_01_26.csv
data/salaries/DKSalaries_2020_03_02.csv
Bad file: data/salaries/DKSalaries_2016_06_13.csv.....
data/salaries/DKSalaries_2018_11_28.csv
data/salaries/DKSalaries_2018_11_14.csv
data/salaries/DKSalaries_2019_10_23.csv


In [112]:
# Construct and save the output
df_res = pd.DataFrame(results, columns=[
    'Date',
    'PG_Name', 'PG_Score', 'PG_Cost',
    'SG_Name', 'SG_Score', 'SG_Cost',
    'SF_Name', 'SF_Score', 'SF_Cost',
    'PF_Name', 'PF_Score', 'PF_Cost',
    'C_Name', 'C_Score', 'C_Cost',
    'G_Name', 'G_Score', 'G_Cost', 
    'F_Name', 'F_Score', 'F_Cost',
    'UTIL_Name', 'UTIL_Score', 'UTIL_Cost',
    'TOT_Score','TOT_Cost'
])
df_res = df_res.sort_values(by=['Date'])
df_res.to_csv('data/DK_Optimal_Scores.csv')